#  Индексация и булев поиск (практика, часть 2)

### Методы для работы с бинарными данными

- модуль struct
- модуль array

### Как компактно хранить бинарные данные
- используя строку (по утверждениям - медленно, т.к. нет append)
- bytearray
- cStringIO - строка с интерфесом файла
- array.array('c') - массив символов

In [ ]:
#!/usr/bin/env python
import random
import timeit

import struct
import array
import cStringIO

ints = []
for x in xrange(10**6):
    ints.append(random.randint(0, 2**31))

def pack_bytearray():
    s = bytearray()
    for x in ints:
        s.extend(struct.pack('I', x))

def pack_array():
    s = array.array('c')
    for x in ints:
        s.extend(struct.pack('I', x))

def pack_str():
    s = ''
    for x in ints:
        s = s + struct.pack('I', x)

def pack_str_io():
    s = cStringIO.StringIO()
    for x in ints:
        s.write(struct.pack('I', x))


def pack_null():
    for x in ints:
        struct.pack('I', x)

args = {
    'setup': 'from __main__ import pack_null,pack_bytearray,pack_str,pack_str_io,pack_array',
    'number': 3
}

null_time = timeit.timeit("pack_null()", **args)
str_time = timeit.timeit("pack_str()", **args)
str_io_time = timeit.timeit("pack_str_io()", **args)
array_time = timeit.timeit("pack_array()", **args)
bytearray_time = timeit.timeit("pack_bytearray()", **args)

print "string approach: %.2f" % (str_time - null_time)
print "cStringIO approach: %.2f" % (str_io_time - null_time)
print "array approach: %.2f" % (array_time - null_time)
print "bytearray approach: %.2f" % (bytearray_time - null_time)

## Разбор дерева запроса

### Алгоритм:
- Находим самый низкоприоритетный оператор
 - наиболее внешний, наиболее правый
 - запомним как token
- Если не нашли => результат = терм | None
- Если нашли:
 - token.left = рекурсивно слева
 - token.right = рекурсивно справа
 - Результатом является token


Необходимо:
- Разобраться в имеющемся коде
- Реализовать части отмеченные 'write your code here'
- Добиться прохождения тестов (тесты корректные!)

In [4]:
import re
import unittest

SPLIT_RGX = re.compile(r'\w+|[\(\)&\|!]', re.U)

class QtreeTypeInfo:
    def __init__(self, value, op=False, bracket=False, term=False):
        self.value = value
        self.is_operator = op
        self.is_bracket = bracket
        self.is_term = term

    def __repr__(self):
        return repr(self.value)

    def __eq__(self, other):
        if isinstance(other, QtreeTypeInfo):
            return self.value == other.value
        return self.value == other


class QTreeTerm(QtreeTypeInfo):
    def __init__(self, term):
        QtreeTypeInfo.__init__(self, term, term=True)


class QTreeOperator(QtreeTypeInfo):
    def __init__(self, val, left=None, right=None):
        QtreeTypeInfo.__init__(self, val, op=True)
        self.priority = get_operator_prio(val)
        self.left = left
        self.right = right


class QTreeBracket(QtreeTypeInfo):
    def __init__(self, bracket, is_open):
        QtreeTypeInfo.__init__(self, bracket, bracket=True)
        self.is_open = is_open


def get_operator_prio(s):
    if s == '|':
        return 0
    if s == '&':
        return 1
    if s == '!':
        return 2

    return None


def is_operator(s):
    return get_operator_prio(s) is not None


def tokenize_query(q):
    tokens = []
    for t in map(lambda w: w.encode('utf-8'), re.findall(SPLIT_RGX, q)):
        if t == '(':
            tokens.append(QTreeBracket(t, True))
        elif t == ')':
            tokens.append(QTreeBracket(t, False))
        elif is_operator(t):
            tokens.append(QTreeOperator(t))
        else:
            tokens.append(QTreeTerm(t))

    return tokens


def build_query_tree(tokens):
    if len(tokens) == 0:
        return None
    if len(tokens) == 1:
        return QTreeTerm(tokens[0])
    if tokens[0].is_bracket and tokens[-1].is_bracket:
        return build_query_tree(tokens[1:-1])
    best_op_pos = -1
    best_op_prio = -1
    best_op_nest = -1
    cur_nest = 0
    for cur_pos, token in enumerate(tokens):
        if token.is_bracket:
            if token.is_open:
                cur_nest += 1
            else:
                cur_nest -= 1
        elif token.is_term:
            continue
        else:
            if cur_nest < best_op_nest and get_operator_prio(token) < best_op_prio:
                best_op_pos = cur_pos
                best_op_nest = cur_nest
                best_op_prio = get_operator_prio(token)
        
    root = QTreeOperator(tokens[best_op_pos],
                         True,
                         build_query_tree(tokens[:best_op_pos]),
                         build_query_tree(tokens[best_op_pos+1:]))


def parse_query(q):
    tokens = tokenize_query(q)
    return build_query_tree(tokens)


""" Collect query tree to sting back. It needs for tests. """
def qtree2str(root, depth=0):
    if root.is_operator:
        need_brackets = depth > 0 and root.value != '!'
        res = ''
        if need_brackets:
            res += '('

        if root.left:
            res += qtree2str(root.left, depth+1)

        if root.value == '!':
            res += root.value
        else:
            res += ' ' + root.value + ' '

        if root.right:
            res += qtree2str(root.right, depth+1)

        if need_brackets:
            res += ')'

        return res
    else:
        return root.value

    
""" Test tokenizer and parser itself """
class QueryParserTest(unittest.TestCase):
    @staticmethod
    def parsed_tree(q):
        return qtree2str(parse_query(q)).decode('utf-8')

    def test_tokenizer(self):
        self.assertEqual(['foxy', '&', 'lady'], tokenize_query('foxy & lady'))
        self.assertEqual(['foxy', '&', 'lady', '|', 'madam'], tokenize_query('foxy & lady | madam'))
        self.assertEqual(['foxy', '&', '(', 'lady', '|', 'madam', ')'], tokenize_query('foxy & (lady | madam)'))
        self.assertEqual(['foxy', '&', '(', '!', 'lady', '|', 'madam', ')'], tokenize_query('foxy & (!lady | madam)'))

    def test_parser(self):
        self.assertEqual('foxy & lady', QueryParserTest.parsed_tree('foxy & lady'))
        self.assertEqual('(foxy & lady) | madam', QueryParserTest.parsed_tree('foxy & lady | madam'))
        self.assertEqual('foxy & (lady | madam)', QueryParserTest.parsed_tree('foxy & (lady | madam)'))

    def test_right_order(self):
        self.assertEqual('((one & two) & three) & four', QueryParserTest.parsed_tree('one & two & three & four'))

    def test_neg(self):
        self.assertEqual('foxy & !(lady | madam)', QueryParserTest.parsed_tree('foxy & !(lady | madam)'))


suite = unittest.TestLoader().loadTestsFromTestCase(QueryParserTest)
unittest.TextTestRunner().run(suite)

EEE.
ERROR: test_neg (__main__.QueryParserTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-4-661939805d9c>", line 155, in test_neg
    self.assertEqual('foxy & !(lady | madam)', QueryParserTest.parsed_tree('foxy & !(lady | madam)'))
  File "<ipython-input-4-661939805d9c>", line 138, in parsed_tree
    return qtree2str(parse_query(q)).decode('utf-8')
  File "<ipython-input-4-661939805d9c>", line 104, in parse_query
    return build_query_tree(tokens)
  File "<ipython-input-4-661939805d9c>", line 98, in build_query_tree
    build_query_tree(tokens[:best_op_pos]),
  File "<ipython-input-4-661939805d9c>", line 98, in build_query_tree
    build_query_tree(tokens[:best_op_pos]),
  File "<ipython-input-4-661939805d9c>", line 98, in build_query_tree
    build_query_tree(tokens[:best_op_pos]),
  File "<ipython-input-4-661939805d9c>", line 98, in build_query_tree
    build_query_tree(tokens[:best_op_pos]),
  Fi

<unittest.runner.TextTestResult run=4 errors=3 failures=0>